In [1]:
import transformers
from transformers import BertModel, BertTokenizerFast, AdamW, get_linear_schedule_with_warmup
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

In [2]:
torch.cuda.empty_cache()


RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0")

In [ ]:
class SentimentClassifier(nn.Module):

    def __init__(self, n_classes):
        super(SentimentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('dbmdz/bert-base-turkish-uncased', return_dict=False)
        self.drop = nn.Dropout(p=0.35)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        output = self.drop(pooled_output)
        output = self.out(output)
        return self.softmax(output)

In [ ]:
model = SentimentClassifier(2)
model.load_state_dict(torch.load("output/model.bin"))
model = model.to(device)

In [10]:
class_names = ["nothing", "provocative"]
def GetTweetSentiment(model, tokenizer, tweet, device):
    encodedTweet = tokenizer.encode_plus(tweet, max_length=128, add_special_tokens=True, pad_to_max_length=True, return_attention_mask=True, return_token_type_ids=False, return_tensors='pt')
    input_ids = encodedTweet["input_ids"].to(device)
    attention_mask = encodedTweet["attention_mask"].to(device)
    output = model(input_ids, attention_mask)
    _, prediction = torch.max(output, dim=1)
    return prediction
    
tokenizer = transformers.BertTokenizerFast.from_pretrained('dbmdz/bert-base-turkish-uncased')


In [23]:
tweet = input("What's the tweet you wanna check?: ")
pred = GetTweetSentiment(model, tokenizer, tweet, device)
print(class_names[pred])

What's the tweet you wanna check?:  Yav he he komplo teorisi bunlar 💦💉🤯 Coronapass’ın ileride derinin altına yerleştirecekleri dijital bir kimliğe dönüşeceğini görmemek için ya beyinsiz olmak lazım ya da işbirlikçi globalist... arası yok! Aptallar ve globalistler köpek gibi çiplenebilir. Teşekkürler ben almiyim!


provocative
